>
>
>  同步历史数据
>
>

In [1]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
tablemap={"banda_emr_etl":"banda"} #,
first=" select 'insert' as kind, '2021-01-01 23:59:59' as etldate,0 as  etlindex, "
end=" , '2021' as year,   '01' as month,  '01' as day from  "
def getTableColum(tableNm,b):
    colum=""
    for index in range(len(b)):
        colum=colum+setDef(b[index]["data_type"],b[index]["col_name"],tableNm)+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col,tableNm):
    if(type[:7] == 'boolean'):
        return "cast ("+table_col+" as string ) as "+table_col
    else:
        return table_col
def make_col_temptable(databaseName,row,stream_tableList):
    tableName=row["tableName"]
#     tableName in stream_tableList and 
    print(tableName)
#      and (tableName=='t_data_upload_log' or tableName=='t_data_upload_loan' or tableName=='t_data_upload_repay' or tableName=='t_data_upload_user')
    if tableName=='t_call_center_info'  :
        tablecolum="desc " +databaseName+"."+tableName;
        tableSchema= spark.sql(tablecolum).collect()
        sql=first+ getTableColum(tableName,tableSchema)+end+databaseName+"."+tableName
        path="s3://rupiahplus-data-warehouse/stream/"+tablemap[databaseName]+"_etl/"+tableName+"/"
    #         print(sql)
        spark.sql(sql).write.mode("append").partitionBy("year","month","day").orc(path)
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate() 
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    df=spark.sql("""show tables in banda_stream_etl""").collect()
    stream_tableList=[]
    for i in df:
        stream_tableList.append(i['tableName'])
    for databaseName in tablemap:
        databasesql="show tables in "+"banda_emr_etl"
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm,stream_tableList) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1629793254659_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t_admin
t_admin_audit
t_admin_group
t_admin_group_rel
t_af_bill
t_ai_rudder_schedule_mobile
t_amount_reduce_record
t_auto_review_loan
t_bankcard
t_bankcard_merchant
t_bankcard_verify
t_brick
t_call_center_info
t_call_center_log
t_case_distribution_log
t_case_reviewer
t_channel_details
t_clear_detail_log
t_code
t_collection_audit
t_collection_blacklist
t_contact
t_coupon_request
t_coupon_send
t_customer
t_customer_app
t_customer_device_info
t_customer_install_info
t_customer_tag_rel
t_data_upload_loan
t_data_upload_log
t_data_upload_repay
t_data_upload_user
t_date_range
t_department_group
t_deposit_callback_log
t_employment
t_engine_rule_detail
t_file
t_fraud_check
t_group
t_hit_blacklist_record
t_invitation
t_job_metric
t_loan_app
t_loan_app_review_summary
t_loan_app_status_log
t_loan_issue
t_loan_log
t_loan_tag_rel
t_login_log
t_lpay
t_lpay_deposit
t_message
t_mobile_tag_rel
t_out_standing_loan_balance
t_password_update_record
t_personal_info
t_record_bankcard
t_record_contact
t_recor

>>>>>>>>>>>>
>>>>>>>>>>>
>>>>>>>>>
step function的代碼
>>>>>>>>>>>
>>>>>>>>>>>

In [2]:
# -*- coding: UTF-8 -*-
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
streamDataBase="banda_stream_etl"
tablemap={"undi":"banda"} #,
# tablemap="#tableMapParams#"
tableList={"banda":["t_admin","t_admin_audit"]} 
# tableList="#tableListParams#"
first=" select 'insert' as kind, '2020-08-01 23:59:59' as etldate,0 as  etlindex, "
end=" , '2020' as year,   '08' as month,  '01' as day from  "
def getTableColum(tableNm,b):
    colum=""
    for index in range(len(b)):
        colum=colum+setDef(b[index]["data_type"],b[index]["col_name"],tableNm)+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col,tableNm):
    if(tableNm=='t_auto_review_loan' and type[:7] == 'boolean'):
        return "cast ("+table_col+" as string ) as "+table_col
    else:
        return table_col
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
#     tablecolum="desc " +databaseName+"."+tableName;
#     tableSchema= spark.sql(tablecolum).collect()
#     sql=first+ getTableColum(tableName,tableSchema)+end+databaseName+"."+tableName
#     path="s3://rupiahplus-data-warehouse/stream/"+tablemap[databaseName]+"_etl/"+tableName+"/"
#     spark.sql(sql).write.mode("append").partitionBy("year","month","day").orc(path)
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        databasesql="show tables in "+streamDataBase
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        ls=[]
        if tablemap[databaseName] in tableList :
            print(tablemap[databaseName])
#             for table in tableList[tablemap[databaseName]]:
#                 if table=='*':
#                     ls =tablelist
#                 else:
#                     ls=ls+[tb for tb in tablelist if tb["tableName"]==table]
#             print(ls)
#             executor=None
#             with futures.ThreadPoolExecutor(max_workers=10) as executor:
#                 futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in ls])
#                 for  future in futures_result[0]:
#                     print(future.result())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

banda

In [1]:
#缺失
t_admin_audit=spark.sql("""
    select
      'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_admin_audit`

""")
t_admin_audit.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_admin_audit/")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1605495800754_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:

t_customer_tag_rel=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_tag_rel`

""")
t_customer_tag_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_tag_rel/")
#   where  id  < 7080681
#无数据
t_record_bankcard=spark.sql("""
    select
    'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_bankcard`
""")
t_record_bankcard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_bankcard/")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1604375599691_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
t_auto_review_loan=spark.sql("""
    select 
    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
    0 as  etlindex,
    id,
    riskengine_id,
    customer_id,
    loan_id,
    name,
    ktp,
    mobile,
    cast(success as string) as success,
    error_message,
    result,
    reject_message,
    create_time,
    finish_time,
    duration,
    real_policy,
    switch_manual_reason,
    raw_data,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_auto_review_loan`
""")
t_auto_review_loan.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_auto_review_loan/")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1605781233008_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:

t_admin=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_admin`
""")
t_admin.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_admin/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

t_admin_group_rel=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_admin_group_rel`
""")
t_admin_group_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_admin_group_rel/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:

t_af_bill=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_af_bill`
""")
t_af_bill.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_af_bill/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:

t_ai_rudder_schedule_mobile=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_ai_rudder_schedule_mobile`
""")
t_ai_rudder_schedule_mobile.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_ai_rudder_schedule_mobile/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
t_bankcard=spark.sql("""
    select 
     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_bankcard`

""")
t_bankcard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_bankcard/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
t_bankcard_verify=spark.sql("""
    select 
     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_bankcard_verify`
""")
t_bankcard_verify.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_bankcard_verify/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
t_bankcard_merchant=spark.sql("""
    select 
     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_bankcard_merchant`
""")
t_bankcard_merchant.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_bankcard_merchant/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
t_channel_details=spark.sql("""
    select   'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_channel_details`
""")
t_channel_details.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_channel_details/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
t_clear_detail_log=spark.sql("""
    select   'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_clear_detail_log`

""")
t_clear_detail_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_clear_detail_log/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
t_code=spark.sql("""
    select   'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_code`

""")
t_code.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_code/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
t_collection_audit=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_collection_audit`

""")
t_collection_audit.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_collection_audit/")
#   where  id < 23830557

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
t_collection_blacklist=spark.sql("""
    select 'insert' as kind,
      '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_collection_blacklist`
""")
t_collection_blacklist.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_collection_blacklist/")
#     where id< 100862

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
t_contact=spark.sql("""
    select    'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_contact`
""")
t_contact.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_contact/")
#  where  id < 23674011

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
t_coupon_send=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_coupon_send`
 
""")
t_coupon_send.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_coupon_send/")
#     where id <25323678

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
t_customer=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer`

""")
t_customer.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer/")
#     where  id<4288305

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
t_customer_app=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_app`
""")
t_customer_app.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_app/")
# where  id  <= 180688

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
###############################################################################################
t_customer_device_info=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_device_info`
""")
t_customer_device_info.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_device_info/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
###############################################################################################
t_customer_install_info=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_install_info`
""")
t_customer_install_info.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_install_info/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
t_customer_tag_rel=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_tag_rel`
""")
t_customer_tag_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_tag_rel/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
t_coupon_request=spark.sql("""
    select    'insert' as kind,
    '2020-12-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '12' as month,
     '01' as day
    from `banda_emr_etl`.`t_coupon_request`
""")
t_coupon_request.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_coupon_request/")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1608545228227_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
t_employment=spark.sql("""
    select  'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_employment`

""")
t_employment.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_employment/")
#   where  id  < 7080681

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
t_engine_rule_detail=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_engine_rule_detail`

""")
t_engine_rule_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_engine_rule_detail/")
#   where  id  < 17301935

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
t_file=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_file`
""")
t_file.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_file/")
#   where  id  < 17301935

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
t_group=spark.sql("""
    select 
     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_group`
""")
t_group.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_group/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
t_hit_blacklist_record=spark.sql("""
    select 
     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_hit_blacklist_record`
""")
t_hit_blacklist_record.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_hit_blacklist_record/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
t_invitation=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_invitation`

""")
t_invitation.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_invitation/")
#    where  id <399154

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
t_job_metric=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_job_metric`



""")
t_job_metric.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_job_metric/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
t_loan_app=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_app`

""")
t_loan_app.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_app/")
#  where  id  < 7052364

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
t_loan_app_review_summary=spark.sql("""
    select 
    'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_app_review_summary`

  
""")
t_loan_app_review_summary.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_app_review_summary/")
#   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
t_loan_app_status_log=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_app_status_log`

""")
t_loan_app_status_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_app_status_log/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
t_loan_issue=spark.sql("""
    select
    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_issue`
     where  id  < 4163212
""")
t_loan_issue.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_issue/")
#    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
t_loan_log=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_log`
""")
t_loan_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_log/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
t_loan_tag_rel=spark.sql("""
    select     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_loan_tag_rel`
""")
t_loan_tag_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_tag_rel/")
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
t_login_log=spark.sql("""
    select     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_login_log`

""")
t_login_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_login_log/")
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
t_lpay=spark.sql("""
    select     'insert' as kind,
   '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_lpay`

""")
t_lpay.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_lpay/")
#    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
t_lpay_deposit=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_lpay_deposit`

""")
t_lpay_deposit.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_lpay_deposit/")
#    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
t_message=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_message`

""")
t_message.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_message/")
# 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
t_mobile_tag_rel=spark.sql("""
    select  'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_mobile_tag_rel`
""")
t_mobile_tag_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_mobile_tag_rel/")
# 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
t_personal_info=spark.sql("""
    select     'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_personal_info`
""")
t_personal_info.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_personal_info/")
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
t_record_bankcard=spark.sql("""
    select     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_bankcard`
""")
t_record_bankcard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_bankcard/")
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
t_record_contact=spark.sql("""
    select    'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_contact`

""")
t_record_contact.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_contact/")
#  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
t_record_employment=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_employment`
""")
t_record_employment.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_employment/")
# 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
t_record_file=spark.sql("""
    select     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_file`
""")
t_record_file.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_file/")
# 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
t_record_personal_info=spark.sql("""
    select 'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_record_personal_info`
""")
t_record_personal_info.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_record_personal_info/")
#     

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
t_review_blacklist=spark.sql("""
    select     'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_review_blacklist`
   
""")
t_review_blacklist.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_review_blacklist/")
#    where  id   <= 101159 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
t_review_step_execution=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_review_step_execution`
""")
t_review_step_execution.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_review_step_execution/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
t_send_coupon_rel=spark.sql("""
    select    'insert' as kind,
   '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_send_coupon_rel`
  
""")
t_send_coupon_rel.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_send_coupon_rel/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
t_sms=spark.sql("""
    select     'insert' as kind,
      '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_sms`
""")
t_sms.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_sms/")
#  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
t_thirdparty_data=spark.sql("""
    select     'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_thirdparty_data`
""")
t_thirdparty_data.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_thirdparty_data/")
#  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
t_tongdun_data=spark.sql("""
    select     'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_tongdun_data`

""")
t_tongdun_data.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_tongdun_data/")
#  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
t_virtual_account=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_virtual_account`
""")
t_virtual_account.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_virtual_account/")
#    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
t_customer_install_info=spark.sql("""
    select    'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '08' as month,
     '01' as day
    from `banda_emr_etl`.`t_customer_install_info`
   
""")
t_customer_install_info.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_customer_install_info/")
#     where  id   < 1833507 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
t_whitelist_old_customer=spark.sql("""
    select    'insert' as kind,
     '2020-02-28 23:59:59' as etldate,
        0 as  etlindex,
    *,
    '2020' as year,
    '02' as month,
    '28' as day
    from `banda_emr_etl`.`t_whitelist_old_customer`
   
""")
t_whitelist_old_customer.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_whitelist_old_customer/")
#     where  id   < 1833507 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"Table or view not found: `banda_emr_etl`.`t_whitelist_old_customer`; line 9 pos 9;\n'Project [insert AS kind#3681, 2020-02-28 23:59:59 AS etldate#3682, 0 AS etlindex#3683, *, 2020 AS year#3684, 02 AS month#3685, 28 AS day#3686]\n+- 'UnresolvedRelation `banda_emr_etl`.`t_whitelist_old_customer`\n"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "Table or view not found: `banda_emr_etl`.`t_whitelist_old_customer`; line 9 pos 9;\n'Project [insert AS kind#3681, 2020-02-28 23:59:59 AS etldate#3682, 0 AS etlin

In [ ]:
t_test_customer=spark.sql("""
    select    'insert' as kind,
     '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
    *,
    '2020' as year,
    '08' as month,
    '01' as day
    from `banda_emr_etl`.`t_test_customer`
   
""")
t_test_customer.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_test_customer/")
#     where  id   < 1833507 

In [4]:
thirdparty_t_task=spark.sql("""
select 
create_time,
 cast(create_time as string) as etldate,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from 
`banda-etl-s3`.`t_auto_review_loan`      a 
 limit 1
""").show(100,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------+----+-----+---+
|create_time            |etldate                |year|month|day|
+-----------------------+-----------------------+----+-----+---+
|2019-12-11 12:21:51.107|2019-12-11 12:21:51.107|2019|12   |11 |
+-----------------------+-----------------------+----+-----+---+

In [6]:
t_loan_app_status_log=spark.sql("""
select  
'insert' as kind,
    '2020-08-01 23:59:59' as etldate,
        0 as  etlindex,
     a.*,
    '2020' as year,
    '' as month,
     '01' as day
from 
`banda_emr_etl`.`t_loan_app_status_log` a 
left join 
  `banda_stream_etl`.`t_loan_app_status_log`  b 
  on a.id =b.id 
  where 1=1
  and b.id is null
""")
#  and a.id>1729793
t_loan_app_status_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/banda_etl/t_loan_app_status_log/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
t_sms=spark.sql("""
select 
count(1)
from 
`banda_emr_etl`.`t_auto_review_loan` a 
left join 
  `banda_stream_etl`.`t_auto_review_loan`  b 
  on a.id =b.id 
  where 1=1
  and b.id is null
""").show()

In [1]:
spark.sql("""select  * from  banda_emr_etl.t_case_distribution_log""").printSchema()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
21,application_1607913558731_0022,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- admin_id: long (nullable = true)
 |-- ref_id: long (nullable = true)
 |-- reviewer_id: long (nullable = true)
 |-- trigger: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- group_id: long (nullable = true)
 |-- create_time: timestamp (nullable = true)

In [17]:
df=spark.sql("""show tables in coupon_stream_etl""").collect()
stream_tableMap=[]
for i in df:
    sql='desc  coupon_stream_etl.'+i['tableName']
    df=spark.sql(sql).collect()
    for j in df:
        if j["data_type"]=='boolean':
            print(i['tableName'],j["data_type"])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…